In [2]:
from sys import prefix
import pandas as pd
import random
import os
import shutil
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import math
from z3 import *

class sudoku:
    
    def __init__(self, init=False, path='benchmarks', solver='sudoku_acs', execl='charts/base.xlsx', opt_path = '/INST_49x49/100'):
        self.path = path
        self.opt_path = path + opt_path
        self.solver = solver
        self.data_path = 'results/' + solver
        self.records = pd.read_excel(execl, header=[0], index_col = [0])
        if init:
            self.records = self.records.drop(index=self.records.index)
            self.get_info(path)
            
    def get_benchmarks(self, percentage=0):
        file_list = os.listdir(self.opt_path)
        target_path = self.opt_path[:-3] + str(percentage)
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        for file in tqdm(file_list, target_path):
            path = os.path.join(self.opt_path, file)
            with open(path) as f:
                grid = []
                order = f.readline()[:-1]
                is_sat = f.readline()
                square = int(order)*int(order)
                for i in range(square):
                    grid.append(f.readline().split())
                index = 0
                total = (100-percentage)*square*square/100
                while index < total:
                    x = random.randint(0, square-1)
                    y = random.randint(0, square-1)
                    if grid[x][y] != '-1':
                        grid[x][y] = '-1'
                        index += 1
                filename = file.split('_')
                filename[1] = str(percentage)
                filename = '_'.join(filename)
                with open(target_path + '/' + filename, 'w') as out:
                    out.write(order + '\n')
                    out.write(is_sat)
                    for i in range(square):
                        line = ''
                        for j in range(square):
                            line = line + grid[i][j] + '\t'
                        out.write(line[:-1] + '\n')

    def get_empty_benchmarks(self, path='benchmarks/INST_64x64/0'):
        square = int(path.split('/')[1].split('x')[-1])
        for i in range(100):
            filepath = f'{path}/inst{str(square)}x{str(square)}_0_{str(i)}.txt'
            # filepath = path + '/inst64x64_0_' + str(i) + '.txt'
            with open(filepath, 'w') as f:
                f.write(str(math.sqrt(square)) + '\n') 
                f.write('1\n')
                for _ in range(square):
                    line = ''
                    for _ in range(square):
                        line = line + '-1' + '\t'
                    f.write(line[:-1] + '\n')

    def get_part_benchmarks(self, number=1, target_path='benchmarks_part'):
        order_list = os.listdir(self.path)
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        for order in order_list:
            order_path = os.path.join(target_path, order)
            if not os.path.exists(order_path):
                os.makedirs(order_path)
            for i in range(0, 105, 5):
                percentage_path = os.path.join(order_path, str(i))
                if not os.path.exists(percentage_path):
                    os.makedirs(percentage_path)
                v1 = order.split('_')[-1]
                v2 = str(i)
                if number != 0:
                    for index in range(number):
                        origin_path = f'benchmarks/inst{v1}_{v2}_{index}.txt'
                        shell = f'cp {origin_path} {percentage_path}'
                        # print(shell)
                        os.system(shell)

    def get_CP_benchmarks(self, path='benchmarks_CP_part'):
        order_list = os.listdir(path)
        for order in order_list:
            order_path = os.path.join(path, order)
            for i in range(0, 105, 5):
                percentage_path = os.path.join(order_path, str(i))
                file_list = os.listdir(percentage_path)
                for file in file_list:
                    file_path = os.path.join(percentage_path, file)
                    target_path = file_path.split('.')[0] + '.dzn'
                    shell = f'mv {file_path} {target_path}'
                    os.system(shell)
                    lines = []
                    with open(target_path) as f:
                        lines = f.readlines()
                    order = int(lines[0][:-1])
                    
                    lines[0] = f'S={str(order)};\n'
                    lines[1] = 'start=[|\n'
                    for i in range(order*order-1):
                        lines[i+2] = lines[i+2].strip('\t\n').replace('\t', ',\t') + '|\n'
                    lines[-1] = lines[-1].strip('\t\n').replace('\t', ',\t') + '|];\n'
                    with open(target_path, 'w') as f:
                        f.writelines(lines)
    
    def get_SMT_benchmarks(self, path='benchmarks_part'):
        shell = f'rm -rf benchmarks_SAT'
        os.system(shell)
        shell = f'cp -rf {path} benchmarks_SAT'
        os.system(shell)
        order_list = os.listdir('benchmarks_SAT')
        for order in order_list:
            order_path = os.path.join('benchmarks_SAT', order)
            origin_path = ''
            for percentage in tqdm(range(0, 105, 5), desc=order_path):
                percentage_path = os.path.join(order_path, str(percentage))
                file_list = os.listdir(percentage_path)
                for file in file_list:
                    if percentage == 0:
                        file_path = os.path.join(percentage_path, file)
                        target_path = file_path.split('.')[0] + '.smt2'
                        origin_path = target_path
                        shell = f'mv {file_path} {target_path}'
                        os.system(shell)
                        lines = []
                        with open(target_path) as f:
                            lines = f.readlines()
                        order = int(lines[0][:-1])
                        n = order*order
                        
                        A = []
                        for i in range(n):
                            A.append(lines[i+2].strip('\t\n').split('\t'))
                        X = [ [ [ Bool("x_%s_%s_%s" % (i+1, j+1, v+1)) for v in range(n) ] for j in range(n) ] for i in range(n) ]
                        domain_c = []
                        for i in range(n):
                            for j in range(n):
                                domain_c += [ AtMost(*[X[i][j][v] for v in range(n)], 1) ]
                                domain_c += [ AtLeast(*[X[i][j][v] for v in range(n)], 1) ] 
                        dists_c = []
                        for i in range(n):
                            for v in range(n):
                                dists_c += [ AtMost(*[X[i][j][v] for j in range(n)], 1) ]
                                dists_c += [ AtLeast(*[X[i][j][v] for j in range(n)], 1) ]
                        for j in range(n):
                            for v in range(n):
                                dists_c += [ AtMost(*[X[i][j][v] for i in range(n)], 1) ]
                                dists_c += [ AtLeast(*[X[i][j][v] for i in range(n)], 1) ]
                        for x in range(order):
                            for y in range(order):
                                for v in range(n):
                                    dists_c += [ AtMost(*[X[x*order + x1][y*order + y1][v] for x1 in range(order) for y1 in range(order)], 1) ]
                                    dists_c += [ AtLeast(*[X[x*order + x1][y*order + y1][v] for x1 in range(order) for y1 in range(order)], 1) ]
                        
                        sudoku_c = domain_c + dists_c
                        s = Solver()
                        s.add(sudoku_c)
                        smt = s.to_smt2().split('\n')
                        smt = [i+'\n' for i in smt]
                        with open(target_path, 'w') as f:
                            f.writelines(smt)
                
                        use_path = file_path.split('.')[0] + '_use.smt2'
                        shell = f'cp {target_path} {use_path}'
                        os.system(shell)
                        
                    else:
                        file_path = os.path.join(percentage_path, file)
                        target_path = file_path.split('.')[0] + '.smt2'
                        shell = f'mv {file_path} {target_path}'
                        os.system(shell)
                        shell = f'cp {origin_path} {target_path}'
                        os.system(shell)
                        use_path = file_path.split('.')[0] + '_use.smt2'
                        shell = f'cp {target_path} {use_path}'
                        os.system(shell)

    def get_SAT_benchmarks(self, path='benchmarks'):
        pass    

    def get_info(self, path):
        folder_list = os.listdir(path)
        for folder in folder_list:
            new_path = os.path.join(path, folder)
            if os.path.isdir(new_path):
                self.get_info(new_path)
            else:
                filename = new_path.split('/')[-1]
                category = filename.split('_')[0]
                percentage = filename.split('_')[1]
                self.records.loc[len(self.records.index)] = [filename, category, percentage]
        
    def save_to(self, filename):
        self.records.to_excel(filename)
        
    def arrange(self):
        # for filename in tqdm(folder_list, desc=folder_path):
        category_list = os.listdir(self.data_path)
        for folder in category_list:
            folder_path = os.path.join(self.data_path, folder)
            file_list = os.listdir(folder_path)
            for file in tqdm(file_list, desc=folder_path):
                path = os.path.join(folder_path, file)
                with open(path) as f:
                    try:
                        if self.solver == 'sudoku_acs':
                            while f.readline():
                                result = f.readline()[:-1]
                                if result == '0':
                                    result = 'sat'
                                elif result == '1':
                                    result = 'timeout'
                                time = f.readline()[:-1]
                                if result == 'timeout':
                                    time = '1000.000000'
                                info = f.readline()
                                # time = info.split(' : ')[-1][:-5]
                                name = info.split(' : ')[0].split('/')[-1]
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_test':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                    f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_lsc':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                    f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_csp':
                            while f.readline():
                                result = f.readline()[:-1]
                                time = '1000.000000'
                                if result[:3] == 'SAT':
                                    result = 'sat'
                                    f.readline()
                                elif result[:3] == '===':
                                    result = 'timeout'
                                info = f.readline()
                                if result == 'sat':
                                    time = int(info.split(' : ')[-1].split(' ')[0]) / 1000.0
                                name = info.split(' : ')[0].split('/')[-1].split('.')[0] + '.txt'
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_sat':
                            while f.readline():
                                info = ''
                                result = f.readline()[:-1]
                                time = '1000.000000'
                                if result[:3] == 's S':
                                    result = 'sat'
                                    info = f.readline()
                                else:
                                    info = result
                                    result = 'timeout'
                                if result == 'sat':
                                    time = int(info.split(' : ')[-1].split(' ')[0]) / 1000.0
                                name = info.split(' : ')[0].split('/')[-1].split('.')[0] + '.txt'
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                    except:
                        print(path)
   

初始时，生成未填充的sudoku，用于求解得到完全填充的sudoku

In [ ]:
# a = sudoku()
# a.get_empty_benchmarks()

初始时，由完全填充的sudoku，删除部分值得到部分填充的benchmarks

In [ ]:
# a = sudoku(opt_path='/INST_81x81/100')
# for i in range(0, 100, 5):
#     a.get_benchmarks(percentage=i)

获取部分benchmarks（每个类型只有部分文件）用于测试

In [ ]:
a = sudoku()
a.get_part_benchmarks(number=0,target_path='benchmarks_SAT')
# for i in range(0, 105, 5):
#     print(i)

获取用于CP的benchmarks

In [ ]:
a = sudoku()
a.get_CP_benchmarks(path='benchmarks_part_CP')

获取用于SAT的benchmarks

In [ ]:
a = sudoku()
a.get_SAT_benchmarks(path='benchmarks_part')

初始化表格

In [ ]:
a = sudoku(init=True)
a.save_to('charts/base.xlsx')

sudoku_lsc 生成表格

In [3]:
b = sudoku(solver='sudoku_lsc')
b.arrange()
b.save_to('charts/sudoku_lsc.xlsx')

results/sudoku_lsc/INST_49x49:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_lsc/INST_9x9:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_lsc/INST_36x36:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_lsc/INST_81x81:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_lsc/INST_64x64:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_lsc/INST_16x16:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_lsc/INST_25x25:   0%|          | 0/21 [00:00<?, ?it/s]

sudoku_acs 生成表格

In [4]:
c = sudoku(solver='sudoku_acs', execl='charts/sudoku_lsc.xlsx')
c.arrange()
c.save_to('charts/sudoku_acs.xlsx')

results/sudoku_acs/INST_49x49:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_acs/INST_9x9:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_acs/INST_36x36:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_acs/INST_81x81:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_acs/INST_81x81/result_40.log
results/sudoku_acs/INST_81x81/result_15.log
results/sudoku_acs/INST_81x81/result_45.log
results/sudoku_acs/INST_81x81/result_90.log
results/sudoku_acs/INST_81x81/result_80.log
results/sudoku_acs/INST_81x81/result_85.log
results/sudoku_acs/INST_81x81/result_25.log
results/sudoku_acs/INST_81x81/result_95.log
results/sudoku_acs/INST_81x81/result_65.log
results/sudoku_acs/INST_81x81/result_30.log
results/sudoku_acs/INST_81x81/result_10.log
results/sudoku_acs/INST_81x81/result_50.log
results/sudoku_acs/INST_81x81/result_5.log
results/sudoku_acs/INST_81x81/result_70.log
results/sudoku_acs/INST_81x81/result_60.log
results/sudoku_acs/INST_81x81/result_75.log
results/sudoku_acs/INST_81x81/result_55.log
results/sudoku_acs/INST_81x81/result_20.log
results/sudoku_acs/INST_81x81/result_100.log
results/sudoku_acs/INST_81x81/result_35.log
results/sudoku_acs/INST_81x81/result_0.log


results/sudoku_acs/INST_64x64:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_acs/INST_16x16:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_acs/INST_25x25:   0%|          | 0/21 [00:00<?, ?it/s]

sudoku_csp 生成表格

In [5]:
c = sudoku(solver='sudoku_csp', execl='charts/sudoku_acs.xlsx')
c.arrange()
c.save_to('charts/sudoku_csp.xlsx')

results/sudoku_csp/INST_49x49:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_csp/INST_9x9:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_csp/INST_36x36:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_csp/INST_81x81:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_csp/INST_64x64:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_csp/INST_16x16:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_csp/INST_25x25:   0%|          | 0/21 [00:00<?, ?it/s]

sudoku_sat 生成表格

In [6]:
c = sudoku(solver='sudoku_sat', execl='charts/sudoku_csp.xlsx')
c.arrange()
c.save_to('charts/sudoku_sat.xlsx')

results/sudoku_sat/INST_49x49:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_sat/INST_9x9:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_sat/INST_36x36:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_sat/INST_64x64:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_sat/INST_16x16:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_sat/INST_25x25:   0%|          | 0/21 [00:00<?, ?it/s]

sudoku_test 生成表格

In [7]:
c = sudoku(solver='sudoku_test', execl='charts/sudoku_sat.xlsx')
c.arrange()
c.save_to('charts/sudoku_test.xlsx')

results/sudoku_test/INST_49x49:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_test/INST_9x9:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_test/INST_36x36:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_test/INST_64x64:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_test/INST_16x16:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_test/INST_25x25:   0%|          | 0/21 [00:00<?, ?it/s]